#Test Model lab

In [1]:
!nvidia-smi

Fri Sep 19 03:15:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn

# Init model
class FaceNet(nn.Module):
    def __init__(self, num_classes):
        super(FaceNet, self).__init__()
        self.fc = nn.Linear(128, num_classes)  # input feature 128

    def forward(self, x):
        return self.fc(x)

# Reload model
num_classes = 10
model = FaceNet(num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/IntelligentSystem/my_model_weights.pth", map_location="cpu"))
model.eval()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class FaceRecognition(nn.Module):
    def __init__(self):
        super(FaceRecognition,self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3,out_channels=8, kernel_size=3, padding=1) #112*112
        self.bn1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, kernel_size=3, padding=1) #56*56
        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=3, padding=1) #28*28
        self.bn3 = nn.BatchNorm2d(32)

        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) #14*14
        self.bn4 = nn.BatchNorm2d(64)

        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) #7*7
        self.bn5 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(in_features=128 * 3 * 3, out_features=512)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.normalize(x, p=2, dim=1)

        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Reload model
model = FaceRecognition()
model.load_state_dict(torch.load("/content/drive/MyDrive/IntelligentSystem/my_model_weights.pth", map_location="cpu"))
model.eval()

Using device: cuda


In [ ]:
from PIL import Image
from torchvision import transforms

# Định nghĩa preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load ảnh
img_path = "test_face.jpg"
image = Image.open(img_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0)  # thêm batch dimension


In [ ]:
with torch.no_grad():
    output = model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

print("Predicted class ID:", predicted_class)


In [ ]:
id2name = {0: "Alice", 1: "Bob", 2: "Charlie"}
print("Người trong ảnh là:", id2name[predicted_class])
